#Criando uma LSTM para NLP na prática

💻 [acessar este jupyter notebook no colab](https://colab.research.google.com/drive/1d5MYs5cEl-55JtO_AmEHIukaKRsY-nwA?usp=sharing)

LSTM, que significa Long Short-Term Memory, é um tipo de Rede Neural Recorrente (RNN) desenvolvida para superar as limitações das RNNs tradicionais.

##Configurações iniciais

* `keras` é uma biblioteca de código aberto em Python que oferece uma interface de alto nível para a construção e treinamento de redes neurais. Foi desenvolvida com o objetivo de facilitar a experimentação rápida e a criação de modelos de aprendizado profundo de maneira simples e eficiente.

In [ ]:
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense
from keras.utils import to_categorical
from keras.initializers import Constant
import os
import zipfile
import urllib.request

* `GloVe` (Global Vectors for Word Representation) é um algoritmo de aprendizado não supervisionado para a geração de representações de palavras (embeddings) a partir de co-ocorrências de palavras em um corpus de texto. Foi desenvolvido por pesquisadores da Universidade de Stanford.

In [ ]:
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
glove_file = "glove.6B.zip"
glove_dir = "glove.6B"
glove_txt = os.path.join(glove_dir, "glove.6B.50d.txt")

if not os.path.exists(glove_txt):
    if not os.path.exists(glove_file):
        print("Baixando GloVe embeddings...")
        urllib.request.urlretrieve(glove_url, glove_file)

    if not os.path.exists(glove_dir):
        os.makedirs(glove_dir)
        print("Extraindo GloVe embeddings...")
        with zipfile.ZipFile(glove_file, 'r') as zip_ref:
            zip_ref.extractall(glove_dir)

Baixando GloVe embeddings...
Extraindo GloVe embeddings...


* A função abaixo abre o arquivo de texto já vetorizado (embeddings)

In [ ]:
def mostrar_primeiras_linhas(caminho_arquivo, num_linhas=5):
    try:
        with open(caminho_arquivo, 'r', encoding='utf-8') as arquivo:
            for i in range(num_linhas):
                linha = arquivo.readline()
                if not linha:
                    break
                print(linha.strip())
    except Exception as e:
        print(f"Erro ao ler o arquivo: {e}")

caminho_arquivo = '/content/glove.6B/glove.6B.200d.txt'
mostrar_primeiras_linhas(caminho_arquivo, num_linhas=50)

the -0.071549 0.093459 0.023738 -0.090339 0.056123 0.32547 -0.39796 -0.092139 0.061181 -0.1895 0.13061 0.14349 0.011479 0.38158 0.5403 -0.14088 0.24315 0.23036 -0.55339 0.048154 0.45662 3.2338 0.020199 0.049019 -0.014132 0.076017 -0.11527 0.2006 -0.077657 0.24328 0.16368 -0.34118 -0.06607 0.10152 0.038232 -0.17668 -0.88153 -0.33895 -0.035481 -0.55095 -0.016899 -0.43982 0.039004 0.40447 -0.2588 0.64594 0.26641 0.28009 -0.024625 0.63302 -0.317 0.10271 0.30886 0.097792 -0.38227 0.086552 0.047075 0.23511 -0.32127 -0.28538 0.1667 -0.0049707 -0.62714 -0.24904 0.29713 0.14379 -0.12325 -0.058178 -0.001029 -0.082126 0.36935 -0.00058442 0.34286 0.28426 -0.068599 0.65747 -0.029087 0.16184 0.073672 -0.30343 0.095733 -0.5286 -0.22898 0.064079 0.015218 0.34921 -0.4396 -0.43983 0.77515 -0.87767 -0.087504 0.39598 0.62362 -0.26211 -0.30539 -0.022964 0.30567 0.06766 0.15383 -0.11211 -0.09154 0.082562 0.16897 -0.032952 -0.28775 -0.2232 -0.090426 1.2407 -0.18244 -0.0075219 -0.041388 -0.011083 0.078186 0.3

In [ ]:
data = """Uma rede neural é uma forma de simular no computador o funcionamento
do cérebro humano"""

In [ ]:
#tokenização do texto, convertendo as palavras em sequencias de inteiros
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])
sequence_data = tokenizer.texts_to_sequences([data])[0]
print(sequence_data)

[1, 2, 3, 4, 1, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [ ]:
#definindo tamanho de vocabulario a ser usado
vocab_size = len(tokenizer.word_index) + 1
seq_length = 5

In [ ]:
#conversão para numpy
sequences = []
for i in range(seq_length, len(sequence_data)):
    seq = sequence_data[i-seq_length:i+1]
    sequences.append(seq)

sequences = np.array(sequences)

print(sequences)

[[ 1  2  3  4  1  5]
 [ 2  3  4  1  5  6]
 [ 3  4  1  5  6  7]
 [ 4  1  5  6  7  8]
 [ 1  5  6  7  8  9]
 [ 5  6  7  8  9 10]
 [ 6  7  8  9 10 11]
 [ 7  8  9 10 11 12]
 [ 8  9 10 11 12 13]
 [ 9 10 11 12 13 14]]


In [ ]:
X, y = sequences[:, :-1], sequences[:, -1]
y = to_categorical(y, num_classes=vocab_size)

print(X)

[[ 1  2  3  4  1]
 [ 2  3  4  1  5]
 [ 3  4  1  5  6]
 [ 4  1  5  6  7]
 [ 1  5  6  7  8]
 [ 5  6  7  8  9]
 [ 6  7  8  9 10]
 [ 7  8  9 10 11]
 [ 8  9 10 11 12]
 [ 9 10 11 12 13]]


* One-hot encoding é uma técnica usada para representar dados categóricos como vetores binários. Cada categoria é convertida em um vetor de bits (0s e 1s), onde apenas a posição correspondente à categoria é marcada com 1, e todas as outras posições são 0.

In [ ]:
#y está no formato One-Hot Encodding
print(y)

[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


* Criação de embedding de forma que ele possa ser utilizado de forma simplificada pelo keras

In [ ]:
embeddings_index = {}
with open(os.path.join('glove.6B', 'glove.6B.50d.txt'), encoding='utf-8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

In [ ]:
#o objetivo dessa função é mostrar como o embedding ficou (estilo dicionário)
def print_primeiros_itens(dicionario, num_itens=5):
    for chave, valor in list(dicionario.items())[:num_itens]:
        print(f"{chave}: {valor}")
print_primeiros_itens(embeddings_index, num_itens=5)

the: [ 4.1800e-01  2.4968e-01 -4.1242e-01  1.2170e-01  3.4527e-01 -4.4457e-02
 -4.9688e-01 -1.7862e-01 -6.6023e-04 -6.5660e-01  2.7843e-01 -1.4767e-01
 -5.5677e-01  1.4658e-01 -9.5095e-03  1.1658e-02  1.0204e-01 -1.2792e-01
 -8.4430e-01 -1.2181e-01 -1.6801e-02 -3.3279e-01 -1.5520e-01 -2.3131e-01
 -1.9181e-01 -1.8823e+00 -7.6746e-01  9.9051e-02 -4.2125e-01 -1.9526e-01
  4.0071e+00 -1.8594e-01 -5.2287e-01 -3.1681e-01  5.9213e-04  7.4449e-03
  1.7778e-01 -1.5897e-01  1.2041e-02 -5.4223e-02 -2.9871e-01 -1.5749e-01
 -3.4758e-01 -4.5637e-02 -4.4251e-01  1.8785e-01  2.7849e-03 -1.8411e-01
 -1.1514e-01 -7.8581e-01]
,: [ 0.013441  0.23682  -0.16899   0.40951   0.63812   0.47709  -0.42852
 -0.55641  -0.364    -0.23938   0.13001  -0.063734 -0.39575  -0.48162
  0.23291   0.090201 -0.13324   0.078639 -0.41634  -0.15428   0.10068
  0.48891   0.31226  -0.1252   -0.037512 -1.5179    0.12612  -0.02442
 -0.042961 -0.28351   3.5416   -0.11956  -0.014533 -0.1499    0.21864
 -0.33412  -0.13872   0.31806   

In [ ]:
#criação de matriz de embedding, cada linha/elemento corresponde a o vetor de embedding de uma palavra do vocabulário
embedding_dim = 50
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

embedding_matrix

array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 4.72

* Agora que já está tudo pronto, o código abaixo cria a rede neural recorrente do tipo LSTM

In [ ]:
#rede neural recorrente LSTM
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, embeddings_initializer=Constant(embedding_matrix), input_length=seq_length, trainable=False))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))

In [ ]:
#compilação da rede neural
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
#treinamento
model.fit(X, y, epochs=300, verbose=2)

Epoch 1/300
1/1 - 3s - loss: 2.6913 - accuracy: 0.1000 - 3s/epoch - 3s/step
Epoch 2/300
1/1 - 0s - loss: 2.5981 - accuracy: 0.2000 - 14ms/epoch - 14ms/step
Epoch 3/300
1/1 - 0s - loss: 2.5071 - accuracy: 0.4000 - 15ms/epoch - 15ms/step
Epoch 4/300
1/1 - 0s - loss: 2.4175 - accuracy: 0.4000 - 14ms/epoch - 14ms/step
Epoch 5/300
1/1 - 0s - loss: 2.3283 - accuracy: 0.5000 - 12ms/epoch - 12ms/step
Epoch 6/300
1/1 - 0s - loss: 2.2389 - accuracy: 0.7000 - 21ms/epoch - 21ms/step
Epoch 7/300
1/1 - 0s - loss: 2.1489 - accuracy: 0.8000 - 21ms/epoch - 21ms/step
Epoch 8/300
1/1 - 0s - loss: 2.0578 - accuracy: 0.8000 - 21ms/epoch - 21ms/step
Epoch 9/300
1/1 - 0s - loss: 1.9655 - accuracy: 0.7000 - 18ms/epoch - 18ms/step
Epoch 10/300
1/1 - 0s - loss: 1.8722 - accuracy: 0.7000 - 16ms/epoch - 16ms/step
Epoch 11/300
1/1 - 0s - loss: 1.7780 - accuracy: 0.7000 - 21ms/epoch - 21ms/step
Epoch 12/300
1/1 - 0s - loss: 1.6832 - accuracy: 0.7000 - 18ms/epoch - 18ms/step
Epoch 13/300
1/1 - 0s - loss: 1.5884 - ac

* A última função é responsável por criar texto completando o texto que for passado.

In [ ]:
def generate_text(seed_text, next_words, model, max_seq_length, diversity=0.7):
    generated_words = []
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_seq_length, padding='pre')
        predicted = model.predict(token_list, verbose=0)[0]

        preds = np.asarray(predicted).astype('float64')
        preds = np.log(preds) / diversity
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)

        probas = np.random.multinomial(1, preds, 1)
        predicted_word_index = np.argmax(probas)

        predicted_word = tokenizer.index_word.get(predicted_word_index, '')
        if predicted_word in generated_words:
            continue
        generated_words.append(predicted_word)
        seed_text += " " + predicted_word
    return seed_text

In [ ]:
seed_text = "Uma rede neural é"
generated_text = generate_text(seed_text, 10, model, seq_length, diversity=0.7)
print(generated_text)

Uma rede neural é forma de simular no computador o funcionamento do cérebro humano
